In [1]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score

In [2]:
tsel = pd.read_csv(r'Prepro_Pelayanan.csv', usecols=['Username','Text','Pelayanan','text_result'])
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [3]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [4]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,admin sinyal telkomsel rumah daerah rawabunga ...


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=1000)
vectors = matrix.fit_transform(tsel['text_result']).toarray()

In [7]:
X = vectors
y = tsel['Pelayanan']

In [38]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.1, random_state=45)

## Klasifikasi NBSVM

In [39]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

In [40]:
x_train_csr = sparse.csr_matrix(x_train)
x_test_csr = sparse.csr_matrix(x_test)
model = NBSVM()

model.fit(x_train_csr,y_train)
y_pred = model.predict(x_test_csr)
y_true = y_test

### random 5

In [11]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 128  106   11]
 [  56 1240   15]
 [  16   73   55]]
              precision    recall  f1-score   support

        -1.0       0.64      0.52      0.58       245
         0.0       0.87      0.95      0.91      1311
         1.0       0.68      0.38      0.49       144

    accuracy                           0.84      1700
   macro avg       0.73      0.62      0.66      1700
weighted avg       0.82      0.84      0.82      1700

Precision:  0.7309557243407373
Recall:  0.6167454306919818
F1:  0.657531565396734
Accuracy:  0.8370588235294117


In [12]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7206480004633505

### random 15

In [16]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 125   91   20]
 [  35 1272   26]
 [  12   63   56]]
              precision    recall  f1-score   support

        -1.0       0.73      0.53      0.61       236
         0.0       0.89      0.95      0.92      1333
         1.0       0.55      0.43      0.48       131

    accuracy                           0.85      1700
   macro avg       0.72      0.64      0.67      1700
weighted avg       0.84      0.85      0.85      1700

Precision:  0.7225898013292751
Recall:  0.637126830873199
F1:  0.6718350027503774
Accuracy:  0.8547058823529412


In [17]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7383900169844018

### random 25

In [26]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 117   90    7]
 [  40 1300   31]
 [  24   49   42]]
              precision    recall  f1-score   support

        -1.0       0.65      0.55      0.59       214
         0.0       0.90      0.95      0.93      1371
         1.0       0.53      0.37      0.43       115

    accuracy                           0.86      1700
   macro avg       0.69      0.62      0.65      1700
weighted avg       0.85      0.86      0.85      1700

Precision:  0.6916046607463491
Recall:  0.620053115496963
F1:  0.6494803993249869
Accuracy:  0.8582352941176471


In [27]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7284372568703524

### random 35

In [36]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 126   81   13]
 [  35 1301   26]
 [  14   60   44]]
              precision    recall  f1-score   support

        -1.0       0.72      0.57      0.64       220
         0.0       0.90      0.96      0.93      1362
         1.0       0.53      0.37      0.44       118

    accuracy                           0.87      1700
   macro avg       0.72      0.63      0.67      1700
weighted avg       0.85      0.87      0.86      1700

Precision:  0.7174465406703096
Recall:  0.6336071836109168
F1:  0.6679152286264255
Accuracy:  0.8652941176470588


In [37]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7376502234332336

### random 45

In [41]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))
print("Precision: ",precision_score(y_true, y_pred, average="macro"))
print("Recall: ",recall_score(y_true, y_pred, average="macro"))
print("F1: ",f1_score(y_true, y_pred,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred))

[[ 118   66   10]
 [  47 1308   26]
 [  13   68   44]]
              precision    recall  f1-score   support

        -1.0       0.66      0.61      0.63       194
         0.0       0.91      0.95      0.93      1381
         1.0       0.55      0.35      0.43       125

    accuracy                           0.86      1700
   macro avg       0.71      0.64      0.66      1700
weighted avg       0.85      0.86      0.86      1700

Precision:  0.7066649524432878
Recall:  0.6357957254940018
F1:  0.6634502153872796
Accuracy:  0.8647058823529412


In [42]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.7374377833749831